# Custom CNN

We'll go over:

* Prepping input
* Sequential model
* Training the model
* Saving the model
* Saving the weights

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.utils import plot_model
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
print("Keras version:", tf.keras.__version__)

Tensorflow version: 2.1.0
Keras version: 2.2.4-tf


## Prep Input

We need to define the following:

* input shape
* training data generator
* validation data generator

Most CNN's depend on a static input size in this example our input will be 160 x 160

Also make sure to notice that we are rescaling the images

In [2]:
input_shape = (160, 160, 3)

num_classes = 10

Here we are creating our training and validation data generators

This should be super simple as long as all of your images are structured correctly

In [3]:
train_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'imagenette-160/train',
    batch_size=64,
    class_mode='categorical',
    target_size=(160, 160)
)

Found 12897 images belonging to 10 classes.


In [4]:
valid_datagen = ImageDataGenerator(rescale=1./255)

valid_generator = valid_datagen.flow_from_directory(
    'imagenette-160/val',
    shuffle=False,
    class_mode='categorical',
    target_size=(160, 160)
)

Found 500 images belonging to 10 classes.


## Sequential Model

There are generally two types of model in the keras library, `Sequential` and `Functional`

The `Sequential` on is easier to instantiate but less flexible when creating the model structure

We'll go over `Functional` models in a later tutorial

### Layers

* Conv2D - Essentially it is just dragging some number of filters M that are of size N x N across the image
* Activation - This is basically always a relu until the final layer a relu is a type of non-linearity
* MaxPooling2D - This will downsample your image taking the highest number out of a N x N filter
* Dropout - Helps regularize your network, basically it helps with overfitting, all nodes will have an X chance of dropping out
* Flatten - This layer will take our 2D network and collapse it down to 1D
* This will add a fully connected layer this is usually done right before the final layer

If these terms confuse you, checkout this playlist from Stanford on Convolutional Neural Networks:

[Link to playlist](https://www.youtube.com/watch?v=ArPaAX_PhIs&list=PLkDaE6sCZn6Gl29AoE31iwdVwSG-KnDzF)

#### Note on the final layer

The final layer uses a `softmax` finction for classification you can use the following table as guideline for your final layer:

| Problem type                            | Last-layer activation | Lost function              |
|-----------------------------------------|-----------------------|----------------------------|
| Binary classification                   | sigmoid               | binary_crossentropy        |
| Multiclass, single-label classification | softmax               | categorical_crossentropy   |
| Multiclass, multiclass classification   | sigmoid               | binary_crossentropy        |
| Regression to arbitrary values          | None                  | mse                        |
| Regression to values between 0 and 1    | sigmoid               | mse or binary_crossentropy |

In [5]:
# instantiate sequential model
model = Sequential()

# first layer always needs input shape but after that it's unecessary
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# notice how the architecture repeats but just adds more filters this is a common scheme used in CNNs
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

We can get a summary of our model with the following function:

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 160, 160, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 160, 160, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 158, 158, 32)      9248      
_________________________________________________________________
activation_1 (Activation)    (None, 158, 158, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 79, 79, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 79, 79, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 79, 79, 64)        1

### Compile the model

* choose and optimizer
* choose a loss function
* define some metrics to monitor

In [7]:
# We'll use RMSProp as our optimizer
opt = RMSprop()

In [8]:
model.compile(loss='categorical_crossentropy',
               optimizer=opt,
               metrics=['accuracy'])

### Fit the model

Here we actually 'fit' or 'train' the model

We need to pass the following:

* training generator (defined in the beginning)
* number of epochs (times you pass through the data)
* validation generator (defined in the beginning)

The best accuracy I was able to achieve without data augmentation was about %72 you can see the biggest jumps between the first 3 epochs

In [10]:
history = model.fit_generator(
    generator=train_generator,
    epochs=3,
    validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 202 steps, validate for 16 steps
Epoch 1/3
202/202 [==============================] - 19s 95ms/step - loss: 0.8218 - accuracy: 0.7388 - val_loss: 0.9120 - val_accuracy: 0.7040
Epoch 2/3
202/202 [==============================] - 19s 96ms/step - loss: 0.6054 - accuracy: 0.8049 - val_loss: 0.9318 - val_accuracy: 0.6960
Epoch 3/3
202/202 [==============================] - 19s 95ms/step - loss: 0.4375 - accuracy: 0.8611 - val_loss: 1.1009 - val_accuracy: 0.7140


## Saving your Model

below we will save both the model itself and the weights used to train it, later we'll show cases where we'll need one or the other

In [10]:
# the most common way to save your model is the following
model.save('models/custom_model.h5')
model.save_weights('models/custom_weights.h5')

In [11]:
# in case you need it as a json
with open('models/custom_architecture.json', 'w') as f:
    f.write(model.to_json())

### Plot your model

You can plot your model to an image to see a rough outline of the graph (we'll view this in tensorboard later)

In [12]:
plot_model(model, to_file='plots/models/custom_model.png')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [19]:
# need
# sudo apt-get install graphviz
# conda install pydot
# conda install pydotplus
# TODO get all packages needed